In [1]:
import torch
import torch.nn as nn
device = "cuda"
device = torch.device(device)
print(device)

c:\Users\mrcgg\anaconda3\envs\ML-pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


In [2]:
# 32 bit rotate left and right
def rotl(x, k):
    return ((x << k) & 0xFFFFFFFF) | (x >> (32 - k))

def rotr(x, k):
    return (x >> k) | ((x << (32 - k) & 0xFFFFFFFF))

assert rotl(0b1100, 1) == 0b11000
assert rotr(0b1100, 1) == 0b0110
assert rotl(2**31, 1) == 1
assert rotr(1, 1) == 2**31
assert rotr(1, 10) == rotl(1, 22)
print("ok")

ok


In [3]:
import torch.nn.functional as F
noise = 0.0
noise_n = 1

train_data = []

# every rotation is possible using only k=1

rot = 1

while len(train_data) < 100000:
    i = torch.randint(0, 2**32, (1,)).item()
    for _ in range(noise_n):
        res = torch.tensor([float(a) for a in bin(rotr(i, rot))[2:].rjust(32,"0")], dtype=torch.float32)
        res = res.to(device)

        b = torch.tensor([float(a) for a in bin(i)[2:].rjust(32,"0")], dtype=torch.float32)
        b += (torch.FloatTensor(32).uniform_(-noise, +noise))
        b = b.to(device)

        train_data.append((b,res))

In [4]:
# for x,y in train_data:
#     if x.shape != torch.Size([32+5]):
#         print(x.shape)
#         print(x,y)
#         break
#     if y.shape != torch.Size([32]):
#         print(y.shape)
#         print(x, y)
#         break
train_data[20]

(tensor([0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 1., 0.,
         0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.],
        device='cuda:0'),
 tensor([0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 1.,
         0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0.],
        device='cuda:0'))

In [5]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np

def get_loaders(train_data, device=device):
    test_size = 0.05
    valid_size = 0.05
    batch_size = 500
    num_workers = 0

    #cuda or cpu
    device = torch.device(device)

    num_train = len(train_data)
    indices = list(range(num_train))
    np.random.shuffle(indices)
    split = int(np.floor(test_size * num_train))
    split2 = int(np.floor((valid_size+test_size) * num_train))
    train_idx, valid_idx, test_idx = indices[split2:], indices[split:split2], indices[:split]

    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)
    test_sampler = SubsetRandomSampler(test_idx)

    # prepare data loaders
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=train_sampler, num_workers=num_workers)
    valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler, num_workers=num_workers)
    test_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=test_sampler, num_workers=num_workers)
    return train_loader, valid_loader, test_loader

train_loader, valid_loader, test_loader = get_loaders(train_data)

In [6]:
import numpy as np
def noise_to_int(bits):
    bits = [round(float(b)) for b in bits]
    bits = "".join([str(b) if b in [0,1] else "0" if b<1/10**5 else "1" for b in bits])
    return int(bits,2)

def lin_to_tuple_32_5(t):
    a = (t[:32],t[32:])
    return tuple([noise_to_int(b) for b in a])

def lin_to_list(t):
    res = []
    for i in range(len(t)//8):
        res += [noise_to_int(t[i*8:i*8+8])]
    return res

In [7]:
X, Y = next(iter(train_loader))

for x,y in zip(X,Y):
    b = noise_to_int(x)
    res = noise_to_int(y)
    # print(bin(b),k,bin(res))
    # print(x)
    # print(y)
    b1 = rotr(b,1)
    assert res == b1
print("ok")

ok


In [11]:
import torch.nn.functional as F

class RotR(nn.Module):
    def __init__(self):     
        super(RotR, self).__init__()   
        self.body = nn.Sequential(
            nn.Linear(32,32),
            # nn.Sigmoid(),
            # nn.Linear(100,32),
            # nn.Sigmoid(),
        )
        
    def forward(self, x):
        out = self.body(x)
        return out

In [12]:
def train(save_file, model, criterion, train_loader, valid_loader, optimizer=None, n_epochs = 100000, f=noise_to_int, lrate=0.005):
    # number of epochs to train the model

    if optimizer is None:
        # specify optimizer (stochastic gradient descent) and learning rate = 0.001
        optimizer = torch.optim.Adam(model.parameters(), lr=lrate)#, weight_decay=0.00000001)

    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf # set initial "min" to infinity
    
    for epoch in range(n_epochs):
        # monitor training loss
        train_loss = 0.0
        valid_loss = 0.0
        results = 0
        results_n = 0
        ###################
        # train the model #
        ###################
        model.train() # prep model for training
        i=0
        for X, target in train_loader:
            i+=1
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            target = target.to(device)
            output = model(X)
            # calculate the loss
            # print(output)
            # print(target)
            loss = criterion(output, target) #
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update running training loss
            train_loss += loss.item()*X.size(0)
            if epoch%100 == 0:
                for x,y in zip(output,target):
                    # print(x.cpu().detach().numpy(),y)
                    a = f(x.cpu().detach().numpy())
                    # a = int(x[0])
                    b = f(y.cpu().detach().numpy())
                    # b = int(y[0])
                    # a = noise_to_int(x)
                    # b = noise_to_int(y)
                    
                    
                    # print(a,b)
                    # print(float(x[0]),float(y[0]))
                    if a==b:
                        
                        results +=1
                    results_n+=1
        ######################    
        # validate the model #
        ######################
        model.eval() # prep model for evaluation
        for X, target in valid_loader:
        
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(X)
            # target = target.to(device)
            # calculate the loss
            loss = criterion(output, target)
            # update running validation loss
            valid_loss += loss.item()*X.size(0)
            

        # print training/validation statistics
        # calculate average loss over an epoch
        train_loss = train_loss/len(train_loader.dataset)
        valid_loss = valid_loss/len(valid_loader.dataset)

        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch+1,
            train_loss,
            valid_loss
            ))

        # save model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                valid_loss_min,
                valid_loss))
            torch.save(model.state_dict(), save_file)
            valid_loss_min = valid_loss
            if train_loss <= 0.000000001:
                print("stop: loss <= 0.00000")
                return
            else:
                print(" loss >= 0.00000")
        
        if results_n != 0 :
            print(f"{results/results_n=}")
            print(f"{results}")
            # if results == results_n and valid_loss <= valid_loss_min:
            #     print("stop: no errors")
            #     return
        

In [14]:
model = RotR().to(device)
model.body[0].weight.data.fill_(0.0)
model.body[0].bias.data.fill_(0.0)
criterion = nn.MSELoss()
train("rotr.pt", model, criterion, train_loader, valid_loader, f=noise_to_int, lrate=0.005)

Epoch: 1 	Training Loss: 0.107173 	Validation Loss: 0.001361
Validation loss decreased (inf --> 0.001361).  Saving model ...
 loss >= 0.00000
results/results_n=0.7717444444444445
69457
Epoch: 2 	Training Loss: 0.007521 	Validation Loss: 0.000046
Validation loss decreased (0.001361 --> 0.000046).  Saving model ...
 loss >= 0.00000
Epoch: 3 	Training Loss: 0.000237 	Validation Loss: 0.000003
Validation loss decreased (0.000046 --> 0.000003).  Saving model ...
 loss >= 0.00000
Epoch: 4 	Training Loss: 0.000024 	Validation Loss: 0.000001
Validation loss decreased (0.000003 --> 0.000001).  Saving model ...
 loss >= 0.00000
Epoch: 5 	Training Loss: 0.000004 	Validation Loss: 0.000000
Validation loss decreased (0.000001 --> 0.000000).  Saving model ...
 loss >= 0.00000
Epoch: 6 	Training Loss: 0.000001 	Validation Loss: 0.000000
Validation loss decreased (0.000000 --> 0.000000).  Saving model ...
 loss >= 0.00000
Epoch: 7 	Training Loss: 0.000000 	Validation Loss: 0.000000
Validation loss dec

In [19]:
model = RotR().to(device)
model.load_state_dict(torch.load("rotr.pt"))

<All keys matched successfully>

In [20]:
model.eval()

results = 0
results_n = 0

for X,Y in train_loader:
    # print(X,Y)
    with torch.no_grad():
        O = model(X)
    for x,y in zip(O,Y):
        a = noise_to_int(x.cpu().detach().numpy())
        b = noise_to_int(y.cpu().detach().numpy())
        # b = int(y[0]) pos_to_int
        # b = noise_to_int(y)
        # print(x,y)
        # print(a,b)
        if a==b:
            
            results +=1
        results_n +=1
print(f"{results/results_n=}")

results/results_n=1.0


In [27]:
model.eval()

results = 0
results_n = 0

for i in range(10000):
    x = torch.randint(0, 2**32, (1,)).item()
    y = rotr(x,1)
    X = torch.tensor([float(a) for a in bin(x)[2:].rjust(32,"0")], dtype=torch.float32).to(device)
    Y = torch.tensor([float(a) for a in bin(y)[2:].rjust(32,"0")], dtype=torch.float32).to(device)
    X = X.unsqueeze(0)
    Y = Y.unsqueeze(0)
    with torch.no_grad():
        O = model(X)
    for x,y in zip(O,Y):
        # print(o,y)
        a = noise_to_int(x.cpu().detach().numpy())
        b = noise_to_int(y.cpu().detach().numpy())
        # b = int(y[0]) pos_to_int
        # b = noise_to_int(y)
        # print(x,y)
        # print(a,b)
        if a==b:
            
            results +=1
        results_n +=1
print(f"{results/results_n=}")

results/results_n=1.0


In [21]:
for layer in model.body:
    if isinstance(layer, nn.Linear):
        print(layer.weight.shape)
        print(layer.bias.shape)
        print(layer.weight)
        print(layer.bias)


torch.Size([32, 32])
torch.Size([32])
Parameter containing:
tensor([[0., 0., 0.,  ..., 0., 0., 1.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0', requires_grad=True)


In [18]:
import torch
state = torch.load("rotr.pt")

for w in state["body.0.weight"]:
    for i in range(len(w)):
        if w[i] > 0:
            w[i] = 1.0
        else:
            w[i] = 0.0

for i in range(len(state["body.0.bias"])):
    state["body.0.bias"][i] = 0.0
torch.save(state,"rotr.pt")